# Graft Loss: Three Dataset Parallel Execution Driver

**Enhanced Pipeline Management for AWS Linux 2023 EC2**

This notebook provides comprehensive management and monitoring for running the graft-loss pipeline across three datasets in parallel:

- **Original Study** (2010-2019) 
- **Full Dataset with COVID** (2010-2023)
- **Full Dataset without COVID** (2010-2019, excluding 2020+)

Key Features:
- 🚀 Parallel execution with resource optimization
- 📊 Real-time monitoring with progress logging
- 💾 Memory optimization for large datasets (1TB RAM support)
- 🎯 Background execution with PID tracking

All logs and step summaries are written to `logs/` by enhanced pipeline logger. Use the cells below to execute runs, then review summaries and monitoring outputs.

## 🧩 Stepwise Pipeline Execution (Manual/Debug)

Use these cells to run each pipeline step individually. This is useful for debugging, validation, or running the pipeline outside the parallel orchestrator. Each cell is idempotent and can be run multiple times.

### Step 0: Set Environment

In [ ]:
# Force-delete entire folder trees (including subfolders and files)
rm_all_dirs <- function(dirs) {
  for (dir_path in dirs) {
    dir_path <- path.expand(dir_path)
    if (dir.exists(dir_path)) {
      message("Removing: ", dir_path)
      unlink(dir_path, recursive = TRUE, force = TRUE)
      message("✅ Removed: ", dir_path)
    } else {
      message("⚠️ Directory not found: ", dir_path)
    }
  }
}

# Your directories - Updated to use current working directory
base <- getwd()  # Use current working directory instead of ~/graft-loss-parallel-processing
dirs <- file.path(base, c("model_data", "logs", "models"))
rm_all_dirs(dirs)




In [ ]:
R.version.string
getRversion()


In [ ]:
# In an R cell in Jupyter notebook
source("scripts/R/check_model_versions.R")

# Check versions
versions <- check_model_versions()
print_model_versions(versions)

# Check compatibility
compatibility <- check_version_compatibility(versions)
if (compatibility$critical_issues) {
  cat("Warnings:\n")
  for (warning in compatibility$warnings) {
    cat(sprintf("  • %s\n", warning))
  }
  cat("\nRecommendations:\n")
  for (rec in compatibility$recommendations) {
    cat(sprintf("  • %s\n", rec))
  }
}

In [ ]:
# run_step_local

suppressPackageStartupMessages({
  library(furrr)
  library(purrr)
  library(withr)
  library(fs)
  library(tibble)
})

# Set number of threads per worker for max CPU utilization
threads_per_worker <- 8  # Adjust as needed for your instance
Sys.setenv(
  OMP_NUM_THREADS = as.character(threads_per_worker),
  MKL_NUM_THREADS = as.character(threads_per_worker),
  OPENBLAS_NUM_THREADS = as.character(threads_per_worker),
  NUMEXPR_NUM_THREADS = as.character(threads_per_worker),
  VECLIB_MAXIMUM_THREADS = as.character(threads_per_worker)
)

cohorts <- list(
    original = list(
      env = list(DATASET_COHORT = "original"),
      log = file.path(getwd(), "logs/orch_bg_original_study.log")
    ),
    full_with_covid = list(
      env = list(DATASET_COHORT = "full_with_covid"), 
      log = file.path(getwd(), "logs/orch_bg_full_with_covid.log")
    ),
    full_without_covid = list(
      env = list(DATASET_COHORT = "full_without_covid"),
      log = file.path(getwd(), "logs/orch_bg_full_without_covid.log")
    )
  )

# Generic runner for local-only steps with proper orch_bg_* logging
run_step_local <- function(step_label, script_rel, cohorts, workers = NULL, seed = TRUE) {
  root <- getwd()
  step_script <- file.path(root, script_rel)
  if (!file.exists(step_script)) stop(sprintf("Script not found: %s", step_script))

  # Forked workers for Linux + local IO
  if (is.null(workers)) workers <- max(1, min(3, parallel::detectCores() - 1))
  plan(multicore, workers = workers)

  on.exit(plan(sequential), add = TRUE)

  results <- future_map(
    names(cohorts),
    function(cohort) {
      cfg <- cohorts[[cohort]]
      env_vars <- c(cfg$env, COHORT_NAME = cohort)
      log_file <- cfg$log

      tryCatch({
        old_wd <- setwd(root); on.exit(setwd(old_wd), add = TRUE)
        
        # Create/ensure log directory exists
        log_dir <- dirname(log_file)
        dir.create(log_dir, showWarnings = FALSE, recursive = TRUE)
        
        # Enhanced logging function
        log_with_resources <- function(message, step = step_label) {
          timestamp <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
          pid <- Sys.getpid()
          
          # Memory usage
          mem_info <- tryCatch({
            if (file.exists("/proc/meminfo")) {
              meminfo <- readLines("/proc/meminfo")
              total_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemTotal", meminfo)])) / 1024 / 1024
              avail_mem <- as.numeric(gsub(".*: *([0-9]+) kB", "\\1", meminfo[grep("MemAvailable", meminfo)])) / 1024 / 1024
              used_mem <- total_mem - avail_mem
              sprintf("MEM: %.1f/%.1f GB (%.1f%%)", used_mem, total_mem, (used_mem/total_mem)*100)
            } else {
              gc_info <- gc()
              sprintf("MEM: %.1f MB", sum(gc_info[,2]))
            }
          }, error = function(e) "MEM: N/A")
          
          # CPU usage
          cpu_info <- tryCatch({
            if (Sys.which("ps") != "") {
              ps_out <- system(sprintf("ps -p %d -o pcpu= 2>/dev/null || echo 'N/A'", pid), intern = TRUE)
              if (length(ps_out) > 0 && ps_out != "N/A") {
                sprintf("CPU: %s%%", trimws(ps_out))
              } else "CPU: N/A"
            } else "CPU: N/A"
          }, error = function(e) "CPU: N/A")
          
          # Format log entry
          step_info <- if (!is.null(step)) sprintf("[STEP: %s] ", step) else ""
          log_entry <- sprintf("[%s] [%s] %s| %s | %s | %s", 
                              timestamp, cohort, step_info, mem_info, cpu_info, message)
          
          # Write to both console and log file
          cat(log_entry, "\n")
          cat(log_entry, "\n", file = log_file, append = TRUE)
        }
        
        # Log step start
        log_with_resources(sprintf("Starting %s", step_label))

        msgs <- character(); wns <- character()
        t0 <- proc.time()
        
        # Capture output and write to log
        output_conn <- file(log_file, open = "a")
        sink(output_conn, split = TRUE)
        sink(output_conn, type = "message", append = TRUE)
        
        on.exit({
          try(sink(type = "message"))
          try(sink())
          try(close(output_conn))
        }, add = TRUE)
        
        withCallingHandlers(
          {
            with_envvar(env_vars, {
              source(step_script, local = new.env(parent = globalenv()))
            })
          },
          message = function(m) msgs <<- c(msgs, conditionMessage(m)),
          warning = function(w) wns  <<- c(wns,  conditionMessage(w))
        )
        elapsed <- as.numeric((proc.time() - t0)[["elapsed"]])
        
        # Log completion
        log_with_resources(sprintf("Completed %s (%.2f seconds)", step_label, elapsed))

        tail_lines <- if (file.exists(log_file)) {
          tryCatch(utils::tail(readLines(log_file, warn = FALSE), 10),
                   error = function(e) sprintf("<could not read log: %s>", e$message))
        } else {
          "<log file not found; run pipeline to generate logs>"
        }

        list(
          cohort   = cohort,
          step     = step_label,
          status   = "ok",
          runtime_s = elapsed,
          warnings = wns,
          messages = msgs,
          log_path = log_file,
          log_tail = tail_lines,
          error    = NULL
        )
      }, error = function(e) {
        # Log error
        tryCatch({
          log_with_resources(sprintf("ERROR in %s: %s", step_label, conditionMessage(e)))
        }, error = function(e2) {
          # Fallback if logging fails
          cat(sprintf("ERROR in %s: %s\n", step_label, conditionMessage(e)), file = log_file, append = TRUE)
        })
        
        list(
          cohort   = cohort,
          step     = step_label,
          status   = "error",
          runtime_s = NA_real_,
          warnings = NULL,
          messages = NULL,
          log_path = log_file,
          log_tail = NULL,
          error    = conditionMessage(e)
        )
      })
    },
    .options = furrr_options(seed = isTRUE(seed))
  )

  # Print concise summary and also return a tibble of results
  cat(sprintf("\n==== Batch Summary (%s) ====\n", step_label))
  walk(results, function(r) {
    cat(sprintf("\n--- %s ---\n", r$cohort))
    cat(sprintf("Status: %s\n", r$status))
    if (!is.null(r$error))   cat(sprintf("Error: %s\n", r$error))
    if (!is.na(r$runtime_s)) cat(sprintf("Runtime (s): %.2f\n", r$runtime_s))
    if (length(r$warnings))  cat(sprintf("Warnings: %s\n", paste(r$warnings, collapse = " | ")))
    if (length(r$messages))  cat(sprintf("Messages: %s\n", paste(r$messages, collapse = " | ")))
    cat(sprintf("Log: %s\n", r$log_path))
    if (!is.null(r$log_tail)) {
      cat("--- Last 10 log lines ---\n")
      cat(paste0(r$log_tail, collapse = "\n"), "\n")
    }
  })
  cat("\n=======================\n")

  # Return structured data for programmatic checks
  as_tibble(map_dfr(results, ~as.list(.x)[c("cohort","step","status","runtime_s","error","log_path")]))
}

# Now you can use run_step_local in your notebook
cat("run_step_local function is now available!\n")

### Step 1: Prepare Data

In [ ]:
# Step 1
run_step_local("STEP 1: Prepare Data", "pipeline/01_prepare_data.R", cohorts)

### Step 2: Resampling 

In [ ]:
# Step 2
run_step_local("STEP 2: Resampling", "pipeline/02_resampling.R", cohorts)

### Step 3: Prepare Model Data

In [ ]:
# Step 3
run_step_local("STEP 3: Prepare Model Data", "pipeline/03_prep_model_data.R", cohorts)

Model (Data Setup, Model Fitting, Aggregate Results)

Settings (recommended for 3 cohorts)

- The model-fitting stage is now split into modular pipeline steps (04–08) to improve debuggability and monitoring:
  - `pipeline/04_data_setup.R` — per-cohort data setup and final dataset preparation
  - `pipeline/05_mc_cv_analysis.R` — Monte Carlo cross-validation analysis and split execution
  - `pipeline/06_parallel_model_fitting.R` — parallel model fitting across splits and models
  - `pipeline/07_model_saving.R` — save final models and create index files
  - `pipeline/08_fallback_handling.R` — fallback handling and recovery steps

Recommended environment settings for per-cohort runs:

- MC_PLAN: `multisession`
- MC_WORKER_THREADS: `1`
- MC_SPLIT_WORKERS per cohort: `floor(available_cores * 0.8 / 3)`

Examples (per-cohort background runs):

```r
# Cohort A (modular pipeline)
Sys.setenv(DATASET_COHORT="full_with_covid", MC_PLAN="multisession", MC_SPLIT_WORKERS="4", MC_WORKER_THREADS="1")
source("pipeline/06_parallel_model_fitting.R")  # runs modularized fitting step (04->08 can be sourced sequentially inside the wrapper)

# Cohort B
Sys.setenv(DATASET_COHORT="original", MC_PLAN="multisession", MC_SPLIT_WORKERS="4", MC_WORKER_THREADS="1")
source("pipeline/06_parallel_model_fitting.R")

# Cohort C
Sys.setenv(DATASET_COHORT="full_without_covid", MC_PLAN="multisession", MC_SPLIT_WORKERS="4", MC_WORKER_THREADS="1")
source("pipeline/06_parallel_model_fitting.R")
```

### Step 4: Data Setup and Preparation

In [ ]:
# Load environment transition utilities
source("scripts/R/environment_transition.R")

# Updated cohorts with proper environment variables for steps 4-8
cohorts_enhanced <- list(
  original = list(
    env = list(
      DATASET_COHORT = "original",
      MC_CV = "1",
      MC_TIMES = "25", 
      USE_ENCODED = "0",
      XGB_FULL = "0",
      USE_CATBOOST = "0",
      FINAL_MODEL_WORKERS = "4",
      FINAL_MODEL_PLAN = "multisession",
      MC_WORKER_THREADS = "8",
      OMP_NUM_THREADS = "1",
      MKL_NUM_THREADS = "1",
      OPENBLAS_NUM_THREADS = "1",
      VECLIB_MAXIMUM_THREADS = "1",
      NUMEXPR_NUM_THREADS = "1"
    ),
    log = file.path(getwd(), "logs/orch_bg_original_study.log")
  ),
  full_with_covid = list(
    env = list(
      DATASET_COHORT = "full_with_covid",
      MC_CV = "1",
      MC_TIMES = "25",
      USE_ENCODED = "0", 
      XGB_FULL = "0",
      USE_CATBOOST = "0",
      FINAL_MODEL_WORKERS = "4",
      FINAL_MODEL_PLAN = "multisession",
      MC_WORKER_THREADS = "8",
      OMP_NUM_THREADS = "1",
      MKL_NUM_THREADS = "1",
      OPENBLAS_NUM_THREADS = "1",
      VECLIB_MAXIMUM_THREADS = "1",
      NUMEXPR_NUM_THREADS = "1"
    ),
    log = file.path(getwd(), "logs/orch_bg_full_with_covid.log")
  ),
  full_without_covid = list(
    env = list(
      DATASET_COHORT = "full_without_covid",
      MC_CV = "1",
      MC_TIMES = "25",
      USE_ENCODED = "0",
      XGB_FULL = "0", 
      USE_CATBOOST = "0",
      FINAL_MODEL_WORKERS = "4",
      FINAL_MODEL_PLAN = "multisession",
      MC_WORKER_THREADS = "8",
      OMP_NUM_THREADS = "1",
      MKL_NUM_THREADS = "1",
      OPENBLAS_NUM_THREADS = "1",
      VECLIB_MAXIMUM_THREADS = "1",
      NUMEXPR_NUM_THREADS = "1"
    ),
    log = file.path(getwd(), "logs/orch_bg_full_without_covid.log")
  )
)


In [ ]:
run_step_local("STEP 4: Data Setup", "pipeline/04_data_setup.R", cohorts)

### Step 5: MC-CV Analysis

In [ ]:
run_step_local("STEP 5: MC-CV Analysis", "pipeline/05_mc_cv_analysis.R", cohorts)

### Step 6: Parallel Model Fitting

In [ ]:
# Step 6: Parallel Model Fitting
run_step_local("STEP 6: Parallel Model Fitting", "pipeline/06_parallel_model_fitting.R", cohorts)


### Step 7 Generate Outputs/C-Index

In [ ]:
# Step 7: Generate Outputs (formerly Step 5)
run_step_local("STEP 7: Generate Outputs", "pipeline/07_generate_outputs.R", cohorts)


### Step 8 📈 C-Index Results and Summary Visualizations

This section summarizes Uno's 1-year C-index across all cohorts and provides a table and plot for model performance comparison.

In [ ]:
readr::read_csv("model_data/model_comparison_metrics.csv")

In [ ]:
# Uno 1-year C-index Summary Table and Plot
suppressPackageStartupMessages({
  library(readr); library(dplyr); library(ggplot2); library(gt); library(here)
})
labels <- c("full","original","covid","full_no_covid")
paths <- setNames(file.path(here::here("data","models"), sprintf("model_mc_summary_%s_uno.csv", labels)), labels)
summaries <- lapply(names(paths)[file.exists(paths)], function(lab){
  readr::read_csv(paths[[lab]], show_col_types = FALSE) %>% mutate(label = lab)
})
res <- bind_rows(summaries) %>%
  mutate(mean = coalesce(mean, uno, AppCindex, AppC, AUC)) %>%
  mutate(se = ifelse(!is.na(sd) & !is.na(n_splits) & n_splits > 1, sd/sqrt(n_splits), NA_real_),
            lwr = ifelse(!is.na(se), mean - 1.96*se, NA_real_),
            upr = ifelse(!is.na(se), mean + 1.96*se, NA_real_)) %>%
  select(label, n_splits, mean, sd, lwr, upr)

# Table
res %>% transmute(Cohort = label, Splits = n_splits,
                  `Uno C (mean)` = round(mean,3),
                  `95% CI` = ifelse(is.na(lwr)|is.na(upr), "—", sprintf("[%.3f, %.3f]", lwr, upr))) %>%
  gt::gt() %>% gt::fmt_missing(everything(), missing_text = "—") %>%
  gt::tab_header(title = gt::md("Uno 1-year C-index by cohort")) %>% print()

# Plot
if (nrow(res) > 0) {
  ggplot(res, aes(x = label, y = mean)) +
    geom_point(size = 3) +
    geom_errorbar(aes(ymin = lwr, ymax = upr), width = 0.15, na.rm = TRUE) +
    coord_cartesian(ylim = c(0.5, 1.0)) +
    labs(x = "Cohort", y = "Uno C-index (1 year)", title = "Uno C-index by cohort (mean ± 95% CI)") +
    theme_minimal(base_size = 12)
} else {
  cat("No C-index summary files found. Run the pipeline to generate results.\n")
}

## Sync Files to S3

In [ ]:
system(paste(
  "aws s3 sync . s3://uva-private-data-lake/graft-loss-parallel-processing/",
  "--delete",
  "--exclude '*checkpoint*'",
  "--exclude '*.tmp'",
  "--exclude '.ipynb_checkpoints/*'"
))
